In [46]:
import logging
import logging.handlers
import os
import io
import pandas as pd
import numpy as np
import requests
import json
# from chessdotcom import get_player_profile, get_player_stats, get_player_game_archives  (not working)
import chess.pgn
from converter.pgn_data import PGNData
import time
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import sys
#import chess_dot_com_api as capi
import requests
from datetime import date
from datetime import datetime
import json
from dateutil.relativedelta import relativedelta
#import googlesheet as g
#import data
#import ratings
from pgn_parser import pgn, parser
import dateutil.relativedelta

In [28]:
student_df = pd.read_csv("../../Desktop/rcc_summer_2024/RCC_2024_summer_student.csv")

In [29]:
student_df.head()

,class,username,real_name
0,Cowboy,oayz01,Aiden Zhang
1,Cowboy,IowaPawn2018,Shilang Zhu
2,Cowboy,Claire_youyou,Claire Yan
3,Cowboy,RayX93,Ray Xia
4,Cowboy,MasonYD,Mason Dong


In [30]:
def last_n_month(n):
    """
    purpose:
    return the month as yyyy/mm format of the past n months from now
    
    input -
    n: number of months from past
    
    output -
    a list of month with yyyy/mm format
    """
    months_lst = []
    for num in range(n):
        months = date.today() + relativedelta(months=-num)
        if months.month <= 9:
            months_lst.append(str(months.year) + "/"+ "0" + str(months.month))
        else:
            months_lst.append(str(months.year) + "/"+ str(months.month))
    return months_lst


def get_archive_games(filename,
                     user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}):
    """
    purpose:
    
    return games in one archive file
    
    input:
    filename - filename that contains game urls
    
    output: 
    """
    games = requests.get(filename,headers = user_agent).json()['games']
    return games

def lowercase_student(student_list):
    """
    to lowercase all the username
    
    input - list, list of student username, regardless upper or lower case
    
    output - list, list of student username, lower case
    """
    
    lower_students = [x.lower() for x in student_list]
    
    return lower_students

In [55]:
def get_user_archives(username, 
                      start_date = '2024-05-04',
                      end_date = '2024-06-17',
                     user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}):
    """
    purpose:
    get archive monthly files of specific chess.com player
    
    input:
    username - username of the chess.com player
    nr_months - integer, nummber of past months that we want to get the archives
    # to request chess.com API
       user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}
    
    output:
    target_month - files of archives according to months parameter
    """
    url = "https://api.chess.com/pub/player/{username}/games/archives".format(username = username)
    archive_request = requests.get(url, headers = user_agent)
    archives = archive_request.json()['archives']
    target_archive = []
    month_format = '%Y-%m-%d'
    start_date = datetime.strptime(start_date, month_format)
    start_date_prev_month = start_date - dateutil.relativedelta.relativedelta(months=1)
    end_date = datetime.strptime(end_date, month_format)
    end_date_post_month = end_date + dateutil.relativedelta.relativedelta(months=1)
    for archive in archives:
        archive_month = str(archive[-7:][:4]) + "-" + str(archive[-7:][5:]) + "-01"
        # convert from string format to datetime format
        datetime_month = datetime.strptime(archive_month, month_format)
        if datetime_month >= start_date_prev_month and datetime_month <= end_date_post_month:
            target_archive.append(archive)
    return target_archive

tianmin_archive = get_user_archives('tianminlyu')

#ray_archive = get_user_archives('rayx93')


tianminlyu


In [56]:
# Latest rating of rapid game, initial setup, rated game
def latest_rating(archives,start_date,end_date):
    for archive in [x for x in archives[::-1]]:
        games = get_archive_games(archive)
        #print(archive)
        #print(len(games))
        for game in games[::-1]:
            end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
            if (end_time <= end_date and end_time >= start_date 
                #and game['initial_setup'] == 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1' 
                and game['rated'] == True 
                and game['time_class'] == 'rapid'):
                #print(end_time)
                if archive.split("/")[5].lower() == game['white']['username'].lower():
                    rating = game['white']['rating']
                else:
                    rating = game['black']['rating']
                return rating
                break
#latest_rating(chesslover, '2024-01-01', '2024-06-12')

# begin rating of rapid game, initial setup, rated game
def begin_rating(archives,start_date,end_date):
    for archive in [x for x in archives]:
        games = get_archive_games(archive)
        #print(archive)
        #print(len(games))
        for game in games:
            end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
            if (end_time <= end_date and end_time >= start_date 
                #and game['initial_setup'] == 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1' 
                and game['rated'] == True 
                and game['time_class'] == 'rapid'):
                #print(end_time)
                #print(game['white']['username'])
                #print(game['black']['username'])
                if archive.split("/")[5].lower() == game['white']['username'].lower():
                    rating = game['white']['rating']
                else:
                    rating = game['black']['rating']
                return rating
                break

def get_student_performance(student_df, start_date= '2024-05-04', end_date= '2024-06-17'):
    class_names = []
    student_usernames = []
    user_realnames = []
    user_begin_ratings = []
    user_latest_ratings = []
    pro_start_time = time.time()
    for i in (range(student_df.shape[0])):
        print(str(i) + "/" + str(student_df.shape[0]))
        class_name = student_df.iloc[i]['class']
        student_username = student_df.iloc[i]['username'].lower()
        print(student_username)
        student_realname = student_df.iloc[i]['real_name']
        archives = get_user_archives(student_username, start_date= start_date, end_date= end_date)
        user_begin_rating = begin_rating(archives, start_date = start_date, end_date = end_date)
        user_latest_rating = latest_rating(archives, start_date = start_date, end_date = end_date)
        
        class_names.append(class_name)
        student_usernames.append(student_username)
        user_realnames.append(student_realname)
        user_begin_ratings.append(user_begin_rating)
        user_latest_ratings.append(user_latest_rating)
        
    pro_end_time = time.time()
    run_time = pro_end_time - pro_start_time
    
    return class_names, student_usernames, user_realnames, user_begin_ratings, user_latest_ratings, run_time

get_student_performance_lst = get_student_performance(student_df)
#print(get_student_performance_lst[5])

0/544
oayz01
oayz01
1/544
iowapawn2018
iowapawn2018
2/544
claire_youyou
claire_youyou
3/544
rayx93
rayx93
4/544
masonyd
masonyd
5/544
chess_zz2023
chess_zz2023
6/544
andrew-wang-2024
andrew-wang-2024
7/544
cateleyn
cateleyn
8/544
danielrxj
danielrxj
9/544
gm_destroyer471_lol
gm_destroyer471_lol
10/544
chess_zz2023
chess_zz2023
11/544
oscar_zh
oscar_zh
12/544
bobogaganoob
bobogaganoob
13/544
ml1363
ml1363
14/544
enzolovechess
enzolovechess
15/544
zhangjunyu2017
zhangjunyu2017
16/544
lucas628shen
lucas628shen
17/544
chenzoe0901
chenzoe0901
18/544
sadiegege
sadiegege
19/544
kevin_tangkc
kevin_tangkc
20/544
17cloud
17cloud
21/544
junjuans1990
junjuans1990
22/544
chloebeat
chloebeat
23/544
jessica-chen
jessica-chen
24/544
albertchw
albertchw
25/544
elisa-wang
elisa-wang
26/544
raphael_fang
raphael_fang
27/544
iriszxr
iriszxr
28/544
vinali1
vinali1
29/544
chickykid1
chickykid1
30/544
lucas101516
lucas101516
31/544
hugoxuedl
hugoxuedl
32/544
gloriawu2023
gloriawu2023
33/544
chiu-ann
chiu-ann


259/544
platothecool1234
platothecool1234
260/544
teresa_long
teresa_long
261/544
mtorm
mtorm
262/544
wzhang2018
wzhang2018
263/544
peridoh
peridoh
264/544
chessn0bb
chessn0bb
265/544
jennayan
jennayan
266/544
martinkingjcs
martinkingjcs
267/544
yushengp
yushengp
268/544
alicecli
alicecli
269/544
maxbest2023
maxbest2023
270/544
albertzxh
albertzxh
271/544
marshallroom
marshallroom
272/544
jiyongjiyong
jiyongjiyong
273/544
nathan1234win
nathan1234win
274/544
blueshadow0
blueshadow0
275/544
luka2021
luka2021
276/544
jingyinghan
jingyinghan
277/544
irene_lin
irene_lin
278/544
tomyliu08
tomyliu08
279/544
anthonychen2017
anthonychen2017
280/544
lightningchen
lightningchen
281/544
getbetterisgood
getbetterisgood
282/544
james12344t
james12344t
283/544
sneakycat123
sneakycat123
284/544
adam_gs
adam_gs
285/544
aaronxiu
aaronxiu
286/544
wiltonwang
wiltonwang
287/544
rcsage
rcsage
288/544
xinyilin
xinyilin
289/544
eddaxiaowang
eddaxiaowang
290/544
ceciliagu123
ceciliagu123
291/544
arthurjin
arth

gavintheking668
516/544
piscescs
piscescs
517/544
brainonfire666
brainonfire666
518/544
pinkprimaryteam
pinkprimaryteam
519/544
clairesister
clairesister
520/544
hardygu666
hardygu666
521/544
littleunicorn2023
littleunicorn2023
522/544
matthewyangscp
matthewyangscp
523/544
cheezycheesechess
cheezycheesechess
524/544
stellal2012
stellal2012
525/544
bigopenplan
bigopenplan
526/544
missnumber6
missnumber6
527/544
owenl5
owenl5
528/544
hollowbill6
hollowbill6
529/544
infinityfinitylol
infinityfinitylol
530/544
thankfuifortune
thankfuifortune
531/544
ihaveyourcat
ihaveyourcat
532/544
jaydenchess1234
jaydenchess1234
533/544
win1031
win1031
534/544
destroyerofyourking
destroyerofyourking
535/544
springmoon0212
springmoon0212
536/544
tinyimagesyz
tinyimagesyz
537/544
sean330
sean330
538/544
pepolireview
pepolireview
539/544
doc-str
doc-str
540/544
n_yu
n_yu
541/544
horseytakesover
horseytakesover
542/544
hhuangterry10
hhuangterry10
543/544
masterguide123
masterguide123


In [57]:
def to_pandas_performance():
    df = pd.DataFrame()
    df['class_name'] = get_student_performance_lst[0]    
    df['username'] = get_student_performance_lst[1]
    df['real_name'] = get_student_performance_lst[2]
    df['begin_rating'] = get_student_performance_lst[3]
    df['latest_rating'] = get_student_performance_lst[4]
    df['change'] = df['latest_rating'] - df['begin_rating']
    #df['start_date'] = start_date
    #df['end_date'] = end_date
    df = df.drop_duplicates()
    return df
df = to_pandas_performance()

In [58]:
df.to_csv("../../Desktop/rcc_summer_2024/RCC_2024_summer_student_performance_output.csv")

In [9]:
tianmin_archive

['https://api.chess.com/pub/player/tianminlyu/games/2024/01',
 'https://api.chess.com/pub/player/tianminlyu/games/2024/02',
 'https://api.chess.com/pub/player/tianminlyu/games/2024/03',
 'https://api.chess.com/pub/player/tianminlyu/games/2024/04',
 'https://api.chess.com/pub/player/tianminlyu/games/2024/05',
 'https://api.chess.com/pub/player/tianminlyu/games/2024/06']

In [10]:
tianmin_archive[0]

'https://api.chess.com/pub/player/tianminlyu/games/2024/01'

In [11]:
def collect_game(username, start_date, end_date):
    usernames = []
    end_times = []
    urls = []
    rateds = []
    time_controls = []
    accuracies = []
    uuids = []
    fens = []
    white_usernames = []
    white_ratings = []
    black_usernames = []
    black_ratings = []
    results = []
    for archive in get_user_archives(username, start_date = start_date, end_date = end_date):
        for game in get_archive_games(archive):
            end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
            if (end_time <= end_date and end_time >= start_date 
                #and game['initial_setup'] == 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1' 
                and game['time_class'] == 'rapid'):
                #print(game.keys())
                url = game['url']
                time_control = game['time_control']
                end_time = game['end_time']
                rated = game['rated']
                try:
                    accuracy = game['accuracies']
                except:
                    accuracy = None
                uuid = game['uuid']
                fen = game['fen']
                white_username = game['white']['username']
                white_rating = game['white']['rating']
                black_username = game['black']['username']
                black_rating = game['black']['rating']
                try:
                    result = game['pgn'].split("Result")[1][2:5]
                except:
                    result = None
                
                usernames.append(username)
                end_times.append(end_time)
                urls.append(url)
                rateds.append(rated)
                time_controls.append(time_control)
                accuracies.append(accuracy)
                uuids.append(uuid)
                fens.append(fen)
                white_usernames.append(white_username)
                white_ratings.append(white_rating)
                black_usernames.append(black_username)
                black_ratings.append(black_rating)
                results.append(result)
                
    df = pd.DataFrame()
    df['username'] = usernames
    df['end_times'] = end_times
    df['urls'] = urls
    df['accuracies'] = accuracies
    df['time_controls'] = time_controls
    df['uuids'] = uuids
    df['rateds'] = rateds
    df['fens'] = fens
    df['white_usernames'] = white_usernames
    df['white_ratings'] = white_ratings
    df['black_usernames'] = black_usernames
    df['black_ratings'] = black_ratings
    df['results'] = results


    #df['start_date'] = start_date
    #df['end_date'] = end_date
    #df = df.drop_duplicates()
    return df

In [12]:
def get_student_game(student_df, start_date= '2024-05-04', end_date= '2024-06-17'):
    get_student_games = []
    
    for i in (range(student_df.shape[0])):
        print(str(i) + "/" + str(student_df.shape[0]))
        student_username = student_df.iloc[i]['username'].lower()
        print(student_username)
        student_game_df = collect_game(student_username, start_date = start_date, end_date = end_date)
        get_student_games.append(student_game_df)
    return get_student_games

student_games = get_student_game(student_df)
student_games_df = pd.concat(student_games)

0/544
1/544
2/544
3/544
4/544
5/544
6/544
7/544
8/544
9/544
10/544
11/544
12/544
13/544
14/544
15/544
16/544
17/544
18/544
19/544
20/544
21/544
22/544
23/544
24/544
25/544
26/544
27/544
28/544
29/544
30/544
31/544
32/544
33/544
34/544
35/544
36/544
37/544
38/544
39/544
40/544
41/544
42/544
43/544
44/544
45/544
46/544
47/544
48/544
49/544
50/544
51/544
52/544
53/544
54/544
55/544
56/544
57/544
58/544
59/544
60/544
61/544
62/544
63/544
64/544
65/544
66/544
67/544
68/544
69/544
70/544
71/544
72/544
73/544
74/544
75/544
76/544
77/544
78/544
79/544
80/544
81/544
82/544
83/544
84/544
85/544
86/544
87/544
88/544
89/544
90/544
91/544
92/544
93/544
94/544
95/544
96/544
97/544
98/544
99/544
100/544
101/544
102/544
103/544
104/544
105/544
106/544
107/544
108/544
109/544
110/544
111/544
112/544
113/544
114/544
115/544
116/544
117/544
118/544
119/544
120/544
121/544
122/544
123/544
124/544
125/544
126/544
127/544
128/544
129/544
130/544
131/544
132/544
133/544
134/544
135/544
136/544
137/544
138/54

In [13]:
#student_games_df.to_csv("../../Desktop/rcc_summer_2024/RCC_2024_summer_student_games.csv")

In [16]:
student_games_df.head()

,username,end_times,urls,accuracies,time_controls,uuids,rateds,fens,white_usernames,white_ratings,black_usernames,black_ratings,results
0,oayz01,1706063489,https://www.chess.com/game/live/99801726395,"{'white': 76.42, 'black': 68.11}",900+10,062f810a-ba5f-11ee-9498-6cfe544c0428,True,1R5N/k4B2/8/4B2p/8/6P1/Q5PP/5K2 b - -,Snakeandhat,306,oayz01,165,1-0
1,oayz01,1706214934,https://www.chess.com/game/live/99871681188,None,600,cc5d5edb-bbc0-11ee-99b6-7dde1001000f,True,1r2kb1r/p1p1p1pp/7n/3p1p2/Q4Pb1/2P4P/PP1PP1P1/...,oayz01,293,pocholo2016,170,1-0
2,oayz01,1706215236,https://www.chess.com/game/live/99871702836,None,600,5f4f9fe7-bbc1-11ee-ae3f-8c0bb801000f,True,rnb1k2r/pp1p2pp/8/1B3p2/3PnB2/2P2P2/PP4PP/RN1Q...,aaalessia00,312,oayz01,205,1-0
3,oayz01,1706667529,https://www.chess.com/game/live/100405903379,"{'white': 26.57, 'black': 28.36}",600,52d3702a-bfdd-11ee-8402-6cfe544c0428,True,2k4Q/5Q2/8/1P6/2P5/8/3B4/4K3 b - -,oayz01,310,chess_ZZ2023,335,1-0
4,oayz01,1707272705,https://www.chess.com/game/live/101010222957,"{'white': 46.47, 'black': 46.88}",900+10,33cfca21-c55d-11ee-aec6-6cfe544c0428,True,2k4r/2p5/8/p1q2br1/P3p3/2P5/2Q3pK/1R2R3 w - -,IowaPawn2018,268,oayz01,372,0-1


In [18]:
student_games_df['results'].isna().sum()

3